In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from collections import Counter 

# Load your dataset
df = pd.read_csv("sentiment_categorized_reviews.csv")
df = df.head(10000)

# Define a function to preprocess text data
def preprocess_text(text):
    # Tokenize words
    tokens = word_tokenize(text.lower())
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words("english")]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(tokens)

# Preprocess text data
df['Cleaned_Text'] = df['Text'].apply(preprocess_text)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Cleaned_Text'], df['Sentiment'], test_size=0.2, random_state=42)

# Initialize VADER SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

# Function to calculate sentiment using VADER
def calculate_vader_sentiment(text):
    sentiment_score = vader_analyzer.polarity_scores(text)
    if sentiment_score['compound'] >= 0.05:
        return 'Positive'
    elif sentiment_score['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Calculate VADER sentiment on test set
vader_predictions = X_test.apply(calculate_vader_sentiment)

# Function to calculate sentiment using SentiWordNet
def calculate_sentiwordnet_sentiment(text):
    pos_score = 0
    neg_score = 0
    tokens = word_tokenize(text.lower())
    for token in tokens:
        synsets = list(swn.senti_synsets(token))
        if synsets:
            synset = synsets[0]  # Take the first synset
            pos_score += synset.pos_score()
            neg_score += synset.neg_score()
    if pos_score > neg_score:
        return 'Positive'
    elif neg_score > pos_score:
        return 'Negative'
    else:
        return 'Neutral'

# Calculate SentiWordNet sentiment on test set
sentiwordnet_predictions = X_test.apply(calculate_sentiwordnet_sentiment)

# Function to calculate sentiment using LIWC (replace with your actual LIWC lexicon)
def calculate_liwc_sentiment(text):
  # Example LIWC categories (replace with actual LIWC categories and their sentiment scores)
  liwc_categories = {
      'positive_emotion': ['happy', 'joy', 'excited'],
      'negative_emotion': ['sad', 'angry', 'disappointed'],
      'positive_social': ['friend', 'love', 'family'],
      'negative_social': ['hate', 'enemy', 'alone'],
  }

  # Count word occurrences in LIWC categories
  category_counts = Counter({category: 0 for category in liwc_categories})
  for token in word_tokenize(text.lower()):
    for category, words in liwc_categories.items():
      if token in words:
        category_counts[category] += 1

  # Calculate sentiment score based on category counts (adjust weights as needed)
  sentiment_score = 0
  sentiment_score += category_counts['positive_emotion'] * 2
  sentiment_score -= category_counts['negative_emotion'] * 1.5
  sentiment_score += category_counts['positive_social'] * 1
  sentiment_score -= category_counts['negative_social'] * 1

  if sentiment_score > 0:
    return 'Positive'
  elif sentiment_score < 0:
    return 'Negative'
  else:
    return 'Neutral'

# Calculate LIWC sentiment on test set
liwc_predictions = X_test.apply(calculate_liwc_sentiment)

# Function to calculate sentiment using General Inquirer (replace with your actual GI lexicon)
def calculate_general_inquirer_sentiment(text):
  # Example General Inquirer categories (replace with actual GI categories and their sentiment scores)
  gi_categories = {
      'positive': ['good', 'great', 'wonderful', 'like'],
      'negative': ['bad', 'terrible', 'horrible', 'dislike'],
      'strong_positive': ['excellent', 'fantastic', 'amazing'],
      'strong_negative': ['awful', 'dreadful', 'appalling'],
  }
# Count word occurrences in GI categories
  category_counts = Counter({category: 0 for category in gi_categories})
  for token in word_tokenize(text.lower()):
    for category, words in gi_categories.items():
      if token in words:
        category_counts[category] += 1

  # Calculate sentiment score based on category counts (adjust weights as needed)
  sentiment_score = 0
  sentiment_score += category_counts['positive'] + category_counts['strong_positive'] * 2
  sentiment_score -= category_counts['negative'] + category_counts['strong_negative'] * 2

  if sentiment_score > 0:
    return 'Positive'
  elif sentiment_score < 0:
    return 'Negative'
  else:
    return 'Neutral'

# Calculate General Inquirer sentiment on test set
general_inquirer_predictions = X_test.apply(calculate_general_inquirer_sentiment)

# Function to evaluate function with improved handling for label distribution
def evaluate_predictions(predictions, true_labels):
  # Calculate evaluation metrics (with zero_division=1)
  accuracy = accuracy_score(true_labels, predictions)
  precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average=None, labels=['Positive', 'Negative', 'Neutral'], zero_division=1)
  return accuracy, precision, recall, f1

# Evaluate VADER predictions with improved evaluation function
vader_accuracy, vader_precision, vader_recall, vader_f1 = evaluate_predictions(vader_predictions, y_test)

# Evaluate SentiWordNet predictions with improved evaluation function
sentiwordnet_accuracy, sentiwordnet_precision, sentiwordnet_recall, sentiwordnet_f1 = evaluate_predictions(sentiwordnet_predictions, y_test)

# Evaluate LIWC predictions with improved evaluation function
liwc_accuracy, liwc_precision, liwc_recall, liwc_f1 = evaluate_predictions(liwc_predictions, y_test)

# Evaluate General Inquirer predictions with improved evaluation function
general_inquirer_accuracy, general_inquirer_precision, general_inquirer_recall, general_inquirer_f1 = evaluate_predictions(general_inquirer_predictions, y_test)

# Display evaluation results
print("VADER Sentiment Analysis:")
print("Accuracy:", vader_accuracy)
print("Precision (Positive, Negative, Neutral):", vader_precision)
print("Recall (Positive, Negative, Neutral):", vader_recall)
print("F1 Score (Positive, Negative, Neutral):", vader_f1)

print("\nSentiWordNet Sentiment Analysis:")
print("Accuracy:", sentiwordnet_accuracy)
print("Precision (Positive, Negative, Neutral):", sentiwordnet_precision)
print("Recall (Positive, Negative, Neutral):", sentiwordnet_recall)
print("F1 Score (Positive, Negative, Neutral):", sentiwordnet_f1)

print("\nLIWC Sentiment Analysis:")
print("Accuracy:", liwc_accuracy)
print("Precision (Positive, Negative, Neutral):", liwc_precision)
print("Recall (Positive, Negative, Neutral):", liwc_recall)
print("F1 Score (Positive, Negative, Neutral):", liwc_f1)

print("\nGeneral Inquirer Sentiment Analysis:")
print("Accuracy:", general_inquirer_accuracy)
print("Precision (Positive, Negative, Neutral):", general_inquirer_precision)A
print("Recall (Positive, Negative, Neutral):", general_inquirer_recall)
print("F1 Score (Positive, Negative, Neutral):", general_inquirer_f1)


VADER Sentiment Analysis:
Accuracy: 0.9375
Precision (Positive, Negative, Neutral): [0.95401662 0.80625    0.68571429]
Recall (Positive, Negative, Neutral): [0.98175599 0.62621359 0.6       ]
F1 Score (Positive, Negative, Neutral): [0.96768755 0.70491803 0.64      ]

SentiWordNet Sentiment Analysis:
Accuracy: 0.793
Precision (Positive, Negative, Neutral): [0.92718142 0.30924855 0.03278689]
Recall (Positive, Negative, Neutral): [0.84207526 0.51941748 0.05      ]
F1 Score (Positive, Negative, Neutral): [0.88258142 0.38768116 0.03960396]

LIWC Sentiment Analysis:
Accuracy: 0.287
Precision (Positive, Negative, Neutral): [0.96052632 0.31325301 0.0267148 ]
Recall (Positive, Negative, Neutral): [0.29133409 0.12621359 0.925     ]
F1 Score (Positive, Negative, Neutral): [0.44706912 0.1799308  0.05192982]

General Inquirer Sentiment Analysis:
Accuracy: 0.616
Precision (Positive, Negative, Neutral): [0.94817814 0.49122807 0.04661017]
Recall (Positive, Negative, Neutral): [0.66761688 0.13592233 0.